# Lab 3.6.5: CrewAI Project - SOLUTIONS

**Complete solutions with explanations and alternative approaches**

---

## Setup

In [ ]:
from typing import Dict, Any, List, Optional, Callable
from dataclasses import dataclass, field
from enum import Enum
import json
from datetime import datetime

print("Setup complete!")

---

## CrewAI Simulation Framework

For learning purposes, we provide a CrewAI-compatible simulation framework.

In [ ]:
@dataclass
class Agent:
    """
    CrewAI-compatible Agent simulation.
    
    Agents are autonomous entities with:
    - Role: Their job/function
    - Goal: What they're trying to achieve
    - Backstory: Context that shapes behavior
    - Tools: Capabilities available to them
    """
    role: str
    goal: str
    backstory: str
    tools: List[Callable] = field(default_factory=list)
    verbose: bool = True
    allow_delegation: bool = False
    memory: bool = True
    
    def __post_init__(self):
        self.task_history: List[str] = []
    
    def execute(self, task_description: str, context: Dict[str, Any] = None) -> str:
        """Execute a task based on agent's role and capabilities."""
        if self.verbose:
            print(f"\n[{self.role}] Executing task...")
            print(f"  Goal: {self.goal}")
        
        # Simulate execution based on role
        result = self._generate_output(task_description, context or {})
        
        if self.memory:
            self.task_history.append(task_description)
        
        return result
    
    def _generate_output(self, task: str, context: Dict) -> str:
        """Generate output based on role."""
        role_lower = self.role.lower()
        
        if "research" in role_lower:
            return f"""Research Findings:
            
1. Key Insight: Based on comprehensive analysis of {task[:50]}...
2. Data Point: Statistical evidence supports the primary hypothesis
3. Expert Opinion: Leading authorities agree on the main conclusions
4. Trend Analysis: Current trajectory indicates continued growth

Sources: Academic journals, industry reports, expert interviews"""
        
        elif "writer" in role_lower or "content" in role_lower:
            return f"""Content Draft:

# {task[:30].title()}...

## Introduction
In today's rapidly evolving landscape, understanding this topic is crucial.

## Key Points
- Primary consideration based on research
- Supporting evidence from multiple sources
- Practical implications for stakeholders

## Conclusion
Moving forward, organizations should prioritize these insights."""
        
        elif "analyst" in role_lower or "strategist" in role_lower:
            return f"""Strategic Analysis:

SWOT Assessment:
- Strengths: Core competencies identified
- Weaknesses: Areas requiring improvement
- Opportunities: Market gaps to exploit
- Threats: Competitive pressures

Recommendations:
1. Short-term: Optimize current operations
2. Medium-term: Expand capabilities
3. Long-term: Transform business model"""
        
        elif "editor" in role_lower or "quality" in role_lower:
            prev_content = context.get("previous_output", "")
            return f"""Edited Content:

Quality Score: 8.5/10

Improvements Made:
- Enhanced clarity and flow
- Corrected grammatical issues
- Strengthened arguments
- Added transitions

Final Version:
{prev_content[:200] if prev_content else 'Polished content ready for publication.'}...

Ready for publication."""
        
        else:
            return f"Completed task: {task}"


@dataclass
class Task:
    """
    CrewAI-compatible Task simulation.
    
    Tasks define:
    - Description: What needs to be done
    - Agent: Who should do it
    - Expected output: Success criteria
    - Context: Dependencies on other tasks
    """
    description: str
    agent: Agent
    expected_output: str = ""
    context: List['Task'] = field(default_factory=list)
    output: Optional[str] = None
    
    def execute(self, crew_context: Dict[str, Any] = None) -> str:
        """Execute the task."""
        # Gather context from dependent tasks
        task_context = crew_context or {}
        
        if self.context:
            previous_outputs = [t.output for t in self.context if t.output]
            task_context["previous_output"] = "\n".join(previous_outputs)
        
        self.output = self.agent.execute(self.description, task_context)
        return self.output


class Process(Enum):
    """Crew execution process types."""
    SEQUENTIAL = "sequential"
    HIERARCHICAL = "hierarchical"


@dataclass
class Crew:
    """
    CrewAI-compatible Crew simulation.
    
    Crews orchestrate:
    - Multiple agents working together
    - Task execution order
    - Context sharing between tasks
    """
    agents: List[Agent]
    tasks: List[Task]
    process: Process = Process.SEQUENTIAL
    verbose: bool = True
    
    def kickoff(self, inputs: Dict[str, Any] = None) -> str:
        """Execute the crew's tasks."""
        if self.verbose:
            print(f"\n{'='*60}")
            print("CREW KICKOFF")
            print(f"{'='*60}")
            print(f"Agents: {[a.role for a in self.agents]}")
            print(f"Tasks: {len(self.tasks)}")
            print(f"Process: {self.process.value}")
        
        results = []
        context = inputs or {}
        
        for i, task in enumerate(self.tasks, 1):
            if self.verbose:
                print(f"\n--- Task {i}/{len(self.tasks)} ---")
                print(f"Description: {task.description[:60]}...")
            
            output = task.execute(context)
            results.append(output)
            
            # Update context with this task's output
            context[f"task_{i}_output"] = output
            
            if self.verbose:
                print(f"Output preview: {output[:100]}...")
        
        final_output = results[-1] if results else "No tasks completed"
        
        if self.verbose:
            print(f"\n{'='*60}")
            print("CREW COMPLETED")
            print(f"{'='*60}")
        
        return final_output


print("CrewAI simulation framework loaded!")

---

## Exercise 1 Solution: Market Research Crew

**Task**: Build a crew that conducts market research and produces a comprehensive report.

In [ ]:
# Define specialized agents
market_researcher = Agent(
    role="Senior Market Research Analyst",
    goal="Conduct comprehensive market research and identify key trends",
    backstory="""You are an experienced market research analyst with 15 years 
    of experience in technology markets. You have a keen eye for identifying 
    emerging trends and understanding competitive landscapes.""",
    verbose=True,
)

competitive_analyst = Agent(
    role="Competitive Intelligence Analyst",
    goal="Analyze competitors and identify market positioning opportunities",
    backstory="""You specialize in competitive analysis with deep expertise in 
    understanding market dynamics, competitor strategies, and finding competitive 
    advantages for clients.""",
    verbose=True,
)

strategy_consultant = Agent(
    role="Strategy Consultant",
    goal="Synthesize research into actionable strategic recommendations",
    backstory="""You are a senior strategy consultant who excels at turning 
    research and analysis into clear, actionable business strategies that 
    drive growth and competitive advantage.""",
    verbose=True,
)

report_writer = Agent(
    role="Executive Report Writer",
    goal="Create polished executive-level reports that communicate insights clearly",
    backstory="""You are an expert business writer who transforms complex 
    analysis into clear, compelling executive reports that drive decision-making.""",
    verbose=True,
)

# Define tasks
research_task = Task(
    description="""Conduct comprehensive market research on the AI assistant market:
    1. Identify market size and growth projections
    2. Map key players and their market share
    3. Analyze technology trends and innovations
    4. Identify customer segments and needs""",
    agent=market_researcher,
    expected_output="Detailed market research report with data and insights",
)

competitive_task = Task(
    description="""Analyze the competitive landscape:
    1. Profile top 5 competitors
    2. Compare product features and pricing
    3. Identify competitive advantages and gaps
    4. Map competitive positioning""",
    agent=competitive_analyst,
    expected_output="Competitive analysis with positioning map",
    context=[research_task],
)

strategy_task = Task(
    description="""Develop strategic recommendations:
    1. Synthesize market and competitive insights
    2. Identify strategic opportunities
    3. Recommend go-to-market approach
    4. Define success metrics""",
    agent=strategy_consultant,
    expected_output="Strategic recommendations with action plan",
    context=[research_task, competitive_task],
)

report_task = Task(
    description="""Create executive summary report:
    1. Compile all research and analysis
    2. Write executive summary
    3. Format for C-suite presentation
    4. Include key metrics and recommendations""",
    agent=report_writer,
    expected_output="Polished executive report ready for presentation",
    context=[research_task, competitive_task, strategy_task],
)

# Create and run the crew
print("=" * 60)
print("MARKET RESEARCH CREW SOLUTION")
print("=" * 60)

market_research_crew = Crew(
    agents=[market_researcher, competitive_analyst, strategy_consultant, report_writer],
    tasks=[research_task, competitive_task, strategy_task, report_task],
    process=Process.SEQUENTIAL,
    verbose=True,
)

result = market_research_crew.kickoff(inputs={"market": "AI Assistant Market"})

print(f"\n--- Final Report ---")
print(result)

---

## Exercise 2 Solution: Content Marketing Crew

**Task**: Build a crew that creates a comprehensive content marketing campaign.

In [ ]:
# Content Marketing Agents
content_strategist = Agent(
    role="Content Strategist",
    goal="Develop comprehensive content strategies aligned with business goals",
    backstory="""You are a seasoned content strategist with expertise in B2B 
    technology marketing. You excel at creating content plans that drive 
    engagement, leads, and conversions.""",
    verbose=True,
)

seo_specialist = Agent(
    role="SEO Specialist",
    goal="Optimize content for search engines and organic visibility",
    backstory="""You are an SEO expert who understands search algorithms, 
    keyword research, and content optimization. You ensure content ranks 
    well and drives organic traffic.""",
    verbose=True,
)

content_writer = Agent(
    role="Senior Content Writer",
    goal="Create engaging, valuable content that resonates with target audiences",
    backstory="""You are an experienced content writer specializing in 
    technology topics. You write clear, engaging content that educates 
    and converts readers.""",
    verbose=True,
)

editor = Agent(
    role="Content Editor",
    goal="Ensure content quality, consistency, and brand alignment",
    backstory="""You are a meticulous editor with a sharp eye for detail. 
    You ensure all content meets the highest quality standards and 
    maintains brand voice consistency.""",
    verbose=True,
)

# Tasks for content creation
strategy_task_content = Task(
    description="""Develop content strategy for AI product launch:
    1. Define target audience personas
    2. Identify content themes and pillars
    3. Create content calendar outline
    4. Define success metrics and KPIs""",
    agent=content_strategist,
    expected_output="Complete content strategy document",
)

seo_task = Task(
    description="""Conduct SEO research and optimization plan:
    1. Research target keywords
    2. Analyze competitor content
    3. Create keyword mapping
    4. Define on-page optimization requirements""",
    agent=seo_specialist,
    expected_output="SEO strategy with keyword recommendations",
    context=[strategy_task_content],
)

writing_task = Task(
    description="""Write flagship blog post for launch:
    1. Create compelling headline
    2. Write engaging introduction
    3. Develop key sections with insights
    4. Include call-to-action""",
    agent=content_writer,
    expected_output="Complete blog post draft",
    context=[strategy_task_content, seo_task],
)

editing_task = Task(
    description="""Edit and finalize content:
    1. Review for clarity and flow
    2. Check grammar and style
    3. Verify SEO optimization
    4. Prepare for publication""",
    agent=editor,
    expected_output="Publication-ready content",
    context=[writing_task],
)

# Create and run the crew
print("\n" + "=" * 60)
print("CONTENT MARKETING CREW SOLUTION")
print("=" * 60)

content_crew = Crew(
    agents=[content_strategist, seo_specialist, content_writer, editor],
    tasks=[strategy_task_content, seo_task, writing_task, editing_task],
    process=Process.SEQUENTIAL,
    verbose=True,
)

content_result = content_crew.kickoff(inputs={"product": "AI Writing Assistant"})

print(f"\n--- Final Content ---")
print(content_result)

---

## Exercise 3 Solution: Product Development Crew

**Task**: Build a crew that plans and documents a new product feature.

In [ ]:
# Product Development Agents
product_manager = Agent(
    role="Product Manager",
    goal="Define product requirements and prioritize features",
    backstory="""You are an experienced product manager with a track record 
    of launching successful products. You excel at understanding customer 
    needs and translating them into product requirements.""",
    verbose=True,
)

ux_designer = Agent(
    role="UX Designer",
    goal="Design intuitive, user-centered experiences",
    backstory="""You are a UX designer passionate about creating delightful 
    user experiences. You combine research, design thinking, and creativity 
    to solve user problems elegantly.""",
    verbose=True,
)

tech_lead = Agent(
    role="Technical Lead",
    goal="Define technical architecture and implementation approach",
    backstory="""You are a senior engineer who bridges product and engineering. 
    You design scalable solutions and guide technical decision-making.""",
    verbose=True,
)

tech_writer = Agent(
    role="Technical Writer",
    goal="Create clear, comprehensive documentation",
    backstory="""You specialize in making complex technical concepts accessible. 
    You create documentation that developers love to use.""",
    verbose=True,
)

# Product Development Tasks
requirements_task = Task(
    description="""Define product requirements for AI chat feature:
    1. Document user stories
    2. Define acceptance criteria
    3. Prioritize features (MVP vs. future)
    4. Identify success metrics""",
    agent=product_manager,
    expected_output="Product requirements document",
)

ux_task = Task(
    description="""Design user experience for AI chat:
    1. Create user flow diagrams
    2. Design key screens/components
    3. Define interaction patterns
    4. Specify accessibility requirements""",
    agent=ux_designer,
    expected_output="UX design specification",
    context=[requirements_task],
)

technical_task = Task(
    description="""Define technical architecture:
    1. Design system architecture
    2. Specify API contracts
    3. Define data models
    4. Identify technical risks and mitigations""",
    agent=tech_lead,
    expected_output="Technical architecture document",
    context=[requirements_task, ux_task],
)

documentation_task = Task(
    description="""Create product documentation:
    1. Write API documentation
    2. Create integration guides
    3. Document configuration options
    4. Write troubleshooting guide""",
    agent=tech_writer,
    expected_output="Complete product documentation",
    context=[requirements_task, technical_task],
)

# Create and run the crew
print("\n" + "=" * 60)
print("PRODUCT DEVELOPMENT CREW SOLUTION")
print("=" * 60)

product_crew = Crew(
    agents=[product_manager, ux_designer, tech_lead, tech_writer],
    tasks=[requirements_task, ux_task, technical_task, documentation_task],
    process=Process.SEQUENTIAL,
    verbose=True,
)

product_result = product_crew.kickoff(inputs={"feature": "AI Chat Assistant"})

print(f"\n--- Final Documentation ---")
print(product_result)

---

## Challenge Solution: Hierarchical Crew with Manager

**Task**: Build a hierarchical crew with a manager agent that coordinates work.

In [ ]:
@dataclass
class ManagerAgent:
    """
    Manager agent that coordinates a team of workers.
    
    Responsibilities:
    - Task decomposition
    - Work assignment
    - Progress monitoring
    - Quality assurance
    """
    role: str = "Project Manager"
    goal: str = "Successfully deliver project by coordinating team efforts"
    
    def __post_init__(self):
        self.team: List[Agent] = []
        self.completed_work: List[Dict] = []
    
    def add_team_member(self, agent: Agent) -> None:
        """Add agent to team."""
        self.team.append(agent)
        print(f"[Manager] Added {agent.role} to team")
    
    def decompose_task(self, task: str) -> List[Dict]:
        """Break down task into subtasks."""
        # Simulated task decomposition
        subtasks = [
            {"description": f"Research phase: {task[:30]}...", "type": "research"},
            {"description": f"Analysis phase: {task[:30]}...", "type": "analysis"},
            {"description": f"Creation phase: {task[:30]}...", "type": "creation"},
            {"description": f"Review phase: {task[:30]}...", "type": "review"},
        ]
        return subtasks
    
    def assign_work(self, subtask: Dict) -> Optional[Agent]:
        """Assign subtask to appropriate team member."""
        task_type = subtask.get("type", "")
        
        for agent in self.team:
            role_lower = agent.role.lower()
            if task_type == "research" and "research" in role_lower:
                return agent
            elif task_type == "analysis" and ("analyst" in role_lower or "strategy" in role_lower):
                return agent
            elif task_type == "creation" and ("writer" in role_lower or "content" in role_lower):
                return agent
            elif task_type == "review" and ("editor" in role_lower or "quality" in role_lower):
                return agent
        
        # Fall back to first available
        return self.team[0] if self.team else None
    
    def review_work(self, work: str) -> Dict:
        """Review completed work."""
        # Simulated review
        return {
            "approved": True,
            "quality_score": 0.85,
            "feedback": "Good work. Minor improvements suggested.",
        }
    
    def coordinate(self, project_goal: str) -> Dict:
        """Coordinate team to complete project."""
        print(f"\n{'='*60}")
        print(f"[MANAGER] Coordinating project: {project_goal[:50]}...")
        print(f"{'='*60}")
        print(f"Team size: {len(self.team)}")
        
        # Decompose
        subtasks = self.decompose_task(project_goal)
        print(f"\n[Manager] Decomposed into {len(subtasks)} subtasks")
        
        results = []
        
        for i, subtask in enumerate(subtasks, 1):
            print(f"\n--- Subtask {i}: {subtask['type'].title()} ---")
            
            # Assign
            assigned_agent = self.assign_work(subtask)
            if not assigned_agent:
                print(f"  ERROR: No agent available")
                continue
            
            print(f"  Assigned to: {assigned_agent.role}")
            
            # Execute
            output = assigned_agent.execute(
                subtask["description"],
                {"previous_outputs": results}
            )
            
            # Review
            review = self.review_work(output)
            print(f"  Review: {'Approved' if review['approved'] else 'Needs revision'} (score: {review['quality_score']:.2f})")
            
            results.append({
                "subtask": subtask,
                "agent": assigned_agent.role,
                "output": output,
                "review": review,
            })
            
            self.completed_work.append(results[-1])
        
        # Final compilation
        print(f"\n[Manager] Compiling final deliverable...")
        
        return {
            "project": project_goal,
            "subtasks_completed": len(results),
            "overall_quality": sum(r["review"]["quality_score"] for r in results) / len(results) if results else 0,
            "results": results,
        }


# Create hierarchical crew
print("=" * 60)
print("HIERARCHICAL CREW SOLUTION")
print("=" * 60)

manager = ManagerAgent(
    role="Senior Project Manager",
    goal="Deliver high-quality project outcomes through effective team coordination"
)

# Add team members
manager.add_team_member(Agent(
    role="Research Specialist",
    goal="Gather and analyze information",
    backstory="Expert researcher with analytical skills",
    verbose=False,
))

manager.add_team_member(Agent(
    role="Strategy Analyst",
    goal="Develop strategic insights",
    backstory="Strategic thinker with business acumen",
    verbose=False,
))

manager.add_team_member(Agent(
    role="Content Writer",
    goal="Create compelling content",
    backstory="Creative writer with expertise in technology",
    verbose=False,
))

manager.add_team_member(Agent(
    role="Quality Editor",
    goal="Ensure content quality",
    backstory="Detail-oriented editor with high standards",
    verbose=False,
))

# Execute project
project_result = manager.coordinate("Create comprehensive guide to implementing AI agents in enterprise")

print(f"\n--- Project Summary ---")
print(f"Subtasks completed: {project_result['subtasks_completed']}")
print(f"Overall quality: {project_result['overall_quality']:.2%}")

---

## Key Takeaways

1. **Role Definition**: Clear roles and backstories improve agent performance
2. **Task Dependencies**: Context passing enables coherent multi-step workflows
3. **Sequential vs Hierarchical**: Choose process based on task complexity
4. **Manager Pattern**: Useful for complex projects needing coordination
5. **Real CrewAI**: Similar patterns work with actual CrewAI library